In [63]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, TensorDataset
import numpy as np
import pandas as pd
from sklearn.metrics import accuracy_score
import nltk
from nltk.tokenize import word_tokenize
from collections import Counter

In [64]:
def label_map(label):
    if label == "neutral":
        return 0
    elif label == "anger":
        return 1
    elif label == "joy":
        return 2
    elif label == "surprise":
        return 3
    elif label == "sadness":
        return 4
    elif label == "disgust":
        return 5
    elif label == "fear":
        return 6

s2idx = {"Chandler":0, "The Interviewer":1, "Joey":2, "Rachel":3, "Monica":4, "Phoebe":5, "Ross":6, "Jade":7, "Mona":8, "Charlie":9}
    
    
def encode(text, word2index, label, N, speaker):
    tokenized = word_tokenize(text)
    encoded = [0]*N
    enc1 = [word2index.get(word) for word in tokenized]
    length = min(N, len(enc1))
    encoded[:length] = enc1[:length]
    if(speaker in s2idx):
        idx = s2idx[speaker]
    else:
        idx = 10
    encoded.insert(0,idx)
    return (encoded,label)

def encode_test(text, word2index, N, speaker):
    tokenized = word_tokenize(text)
    for i,word in enumerate(tokenized):
        if word2index.get(word)==None:
            tokenized[i]='unk'

    encoded = [0]*N
    enc1 = [word2index.get(word) for word in tokenized]

    length = min(N, len(enc1))
    encoded[:length] = enc1[:length]
    if(speaker in s2idx):
        idx = s2idx[speaker]
    else:
        idx = 10
    encoded.insert(0,idx)
    return encoded


def train(model, iterator, optimizer, criterion):
    epoch_loss = 0
    epoch_acc = 0

    model.train()
    for batch in iterator:
        optimizer.zero_grad()
        text = batch[0].to(device)
        target = batch[1]
        target = target.type(torch.LongTensor)
        target = target.to(device)
        preds = model(text)
        loss = criterion(preds, target)
        _, pred = torch.max(preds, 1)
        acc = accuracy_score(pred.tolist(), target.tolist())

        loss.backward()
        optimizer.step()

        epoch_loss += loss.item()
        epoch_acc += acc

    return epoch_loss / len(iterator), epoch_acc / len(iterator)

def evaluate(model, iterator, criterion):
    epoch_loss = 0
    epoch_acc = 0

    model.eval()
    for batch in iterator:
        text = batch[0].to(device)
        target = batch[1]
        target = target.type(torch.LongTensor)
        target = target.to(device)
        preds = model(text)
        loss = criterion(preds, target)
        _, pred = torch.max(preds, 1)
        acc = accuracy_score(pred.tolist(), target.tolist())

        epoch_loss += loss.item()
        epoch_acc += acc

    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [65]:
train_df = pd.read_csv('train_HW2dataset.csv')
dev_df = pd.read_csv('dev_HW2dataset.csv')

train_df=train_df[['Emotion','Utterance','Speaker']]
dev_df=dev_df[['Emotion','Utterance','Speaker']]

train_set = list(train_df.to_records(index=False))
dev_set = list(dev_df.to_records(index=False))

counts = Counter()
for ds in [train_set, dev_set]:
    for label,text,speaker in ds:
        counts.update(word_tokenize(text))

word2index = {'unk':0}
for i,word in enumerate(counts.keys()):
    word2index[word] = i+1
index2word = {v:k for k,v in word2index.items()}

train_encoded = [(encode(Utterance,word2index,label_map(label),12,s)) for label, Utterance, s in train_set]
dev_encoded   = [(encode(Utterance,word2index,label_map(label),12,s)) for label, Utterance,s in dev_set]

train_x = np.array([tweet for tweet, label in train_encoded])
train_y = np.array([label for tweet, label in train_encoded])
dev_x = np.array([tweet for tweet, label in dev_encoded])
dev_y = np.array([label for tweet, label in dev_encoded])

batch_size = 32

train_ds = TensorDataset(torch.from_numpy(train_x), torch.from_numpy(train_y))
dev_ds = TensorDataset(torch.from_numpy(dev_x), torch.from_numpy(dev_y))

train_dl = DataLoader(train_ds, shuffle=True, batch_size=batch_size, drop_last=True)
dev_dl = DataLoader(dev_ds, shuffle=True, batch_size=batch_size, drop_last=True)


In [86]:
total_x = np.append(train_x, dev_x, axis=0)
total_y = np.append(train_y, dev_y, axis=0)

In [68]:
src_vocab_size = len(word2index)
dimension_model = 128
num_layers = 5
hidden_size = 60
linear_hidden_size = 30
classes = 7
dropout = 0.2        
lr = 1e-3

In [69]:
class LSTM(torch.nn.Module):
    def __init__(self):
        super(LSTM, self).__init__()
        self.embed = torch.nn.Embedding(src_vocab_size, dimension_model)                        
        self.lstm = torch.nn.LSTM(input_size=dimension_model, hidden_size=hidden_size,num_layers=num_layers,dropout=dropout)
        self.linear = torch.nn.Linear(hidden_size, linear_hidden_size)
        self.linear1 = torch.nn.Linear(linear_hidden_size, classes)
    def forward(self,data):
        x = self.embed(data)                          
        x,(h_n, c_n) = self.lstm(x.transpose(0, 1))  
                                                         
                                                    
        x = self.linear(x[-1])                          
        x = self.linear1(x)                            
        return x

In [70]:
import os
os.environ['CUDA_LAUNCH_BLOCKING'] = "1"

In [89]:
k=5
def cv_idx():
    idxarr = np.arange(len(total_y))
    np.random.shuffle(idxarr)
    Q = int( len(total_y) / k )
    #print(Q)
    rem = len(total_y) % k
    
    ret = [[] for _ in range(k)]
    foldsize = np.zeros(k)
    for i in range(k):
        if k < rem:
            foldsize[i] = Q + 1
        else:
            foldsize[i] = Q
    for i in range(k):
        train = []
        val = []
        start = 0
        for j in range(k): 
            if j == i:
                for s in range(int(foldsize[j])):
                    val.append(idxarr[start])
                    start += 1
            else:
                for s in range(int(foldsize[j])):
                    train.append(idxarr[start])
                    start += 1
        ret[i].append(np.array(train))
        ret[i].append(np.array(val))
    return ret

kfold_data = cv_idx()

def cross_val(i):
    batch_size = 32
    train_x = total_x[kfold_data[i][0]]
    train_y = total_y[kfold_data[i][0]]
    dev_x = total_x[kfold_data[i][1]]
    dev_y = total_y[kfold_data[i][1]]
    train_ds = TensorDataset(torch.from_numpy(train_x), torch.from_numpy(train_y))
    dev_ds = TensorDataset(torch.from_numpy(dev_x), torch.from_numpy(dev_y))

    train_dl = DataLoader(train_ds, shuffle=True, batch_size=batch_size, drop_last=True)
    dev_dl = DataLoader(dev_ds, shuffle=True, batch_size=batch_size, drop_last=True)
    return train_dl, dev_dl

In [90]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu') 

model = LSTM().to(device)                           
criterion = torch.nn.CrossEntropyLoss()                  
optimizer = torch.optim.Adam(model.parameters(),lr=lr) 

best_acc = 0

for epoch in range(10):
    total_t_loss = 0
    total_v_loss = 0
    total_t_acc = 0
    total_v_acc = 0
    for i in range(k):
        train_dl, dev_dl = cross_val(i)
        train_loss, train_acc = train(model, train_dl,
                                    optimizer, criterion)
        valid_loss, valid_acc = evaluate(model, dev_dl,
                                        criterion)
        total_t_loss += train_loss
        total_v_loss += valid_loss
        total_t_acc += train_acc
        total_v_acc += valid_acc
    total_t_loss /= k
    total_v_loss /= k
    total_t_acc /= k
    total_v_acc /= k

    print(f'Epoch: {epoch+1:02}, Train Loss: {total_t_loss:.3f}, Train Acc: {total_t_acc * 100:.2f}%, Val. Loss: {total_v_loss:.3f}, Val. Acc: {total_v_acc * 100:.2f}%')    
    if best_acc <= total_v_acc:
        best_acc = total_v_acc
        PATH=f"epoch{epoch+1}_val.accuracy{total_v_acc:.3f}%.pt"
        torch.save({
                'epoch': epoch+1,
                'model_state_dict': model.state_dict(),
                'optimizer_state_dict': optimizer.state_dict(),
                'loss': total_v_acc,
                }, PATH)

Epoch: 01, Train Loss: 1.309, Train Acc: 55.11%, Val. Loss: 1.265, Val. Acc: 56.71%
Epoch: 02, Train Loss: 0.907, Train Acc: 70.21%, Val. Loss: 0.856, Val. Acc: 72.30%
Epoch: 03, Train Loss: 0.594, Train Acc: 81.50%, Val. Loss: 0.549, Val. Acc: 82.96%
Epoch: 04, Train Loss: 0.430, Train Acc: 86.47%, Val. Loss: 0.387, Val. Acc: 87.68%
Epoch: 05, Train Loss: 0.328, Train Acc: 89.59%, Val. Loss: 0.285, Val. Acc: 90.64%
Epoch: 06, Train Loss: 0.266, Train Acc: 91.49%, Val. Loss: 0.227, Val. Acc: 92.74%
Epoch: 07, Train Loss: 0.226, Train Acc: 92.69%, Val. Loss: 0.197, Val. Acc: 93.56%
Epoch: 08, Train Loss: 0.191, Train Acc: 93.79%, Val. Loss: 0.149, Val. Acc: 95.22%
Epoch: 09, Train Loss: 0.164, Train Acc: 94.79%, Val. Loss: 0.132, Val. Acc: 95.82%
Epoch: 10, Train Loss: 0.147, Train Acc: 95.22%, Val. Loss: 0.116, Val. Acc: 96.21%


In [91]:
test_df = pd.read_csv('test_HW2dataset.csv')
test_df=test_df[['Utterance','Speaker']]
#test_set = test_df.values.tolist()
test_set = list(test_df.to_records(index=False))
#print(test_set)
test_encoded=[]
test_encoded+=[encode_test(Utterance, word2index, 10, s) for Utterance,s in test_set]
test_x = np.array(test_encoded)
test_ds = TensorDataset(torch.from_numpy(test_x))
test_dl = DataLoader(test_ds, shuffle=False)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu') 
model = LSTM().to(device)                               
criterion = torch.nn.CrossEntropyLoss()                  
optimizer = torch.optim.Adam(model.parameters(),lr=lr) 
checkpoint = torch.load(PATH)
model.load_state_dict(checkpoint['model_state_dict'])
optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
epoch = checkpoint['epoch']

model.eval()
predict=[]
for deta in test_dl:
    text = deta[0].to(device)
    preds = model(text)
    _, pred = torch.max(preds, 1)
    predict.append(pred.item())

In [92]:
print(len(predict))
ans = [[i, pre] for [i,pre] in enumerate(predict)]


3400


In [93]:
import csv
h = ['index', 'emotion']
with open('predict.csv', 'w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(h)
    writer.writerows(ans)